In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils_cd import (
        split_dataset,
        standard_deviation,
        plot_comparison_results,
        impute_values,
        plot_results,
        plot_param_improv,
        plot_results_from_csv,
        aps_classifier_statistics
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA

pd.options.display.max_columns = None
sns.set(style='darkgrid')

CLASS = 'class'
train = pd.read_csv('./aps_failure_training_set.csv',
                        skiprows=20,keep_default_na=False, na_values='na')
test = pd.read_csv('./aps_failure_test_set.csv',
                        skiprows=20,keep_default_na=False, na_values='na')

X_train, y_train = split_dataset(train, CLASS)
X_test, y_test = split_dataset(test, CLASS)
y_train = y_train.map({'pos': 1, 'neg': 0})
y_test = y_test.map({'pos': 1, 'neg': 0})

def normalize(X_train, X_test):
    normalizer = Normalizer().fit(X_train)

    X_train_norm = normalizer.transform(X_train)
    X_test_norm = normalizer.transform(X_test)
    
    return X_train_norm, X_test_norm

aps = pd.concat([X_train, X_test])

clf = KNeighborsClassifier()
results = {}
res_i = 0

In [10]:
X_train, X_test = X_train.fillna(X_train.mean()), X_test.fillna(X_train.mean())

In [11]:
res = aps_classifier_statistics(clf, X_train, X_test, y_train, y_test)
results[res_i] = {'Price': res['score'], 'Transformation': 'Baseline'}
res_i += 1
print(res)

{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.983875, 'confusion_matrix': array([[15574,    51],
       [  207,   168]]), 'sensibility': 0.448, 'specificity': 0.996736, 'score': 104010}


In [13]:
balancers = [(RandomUnderSampler(random_state=42), 'Undersample 50-50'), (RandomUnderSampler(ratio=0.3, random_state=42), 'Undersample 70-30'), (SMOTE(ratio=1.0, random_state=42), 'SMOTE 50-50'), (SMOTE(ratio=0.3, random_state=42), 'SMOTE 70-30')]
filename = 'nb_balancing'
measures = {}
i = 0

for balancer, name in balancers:
    X_train_bal, y_train_bal = balancer.fit_sample(X_train, y_train)
    res = aps_classifier_statistics(clf, X_train_bal, X_test, y_train_bal, y_test)
    
    measures[i] = {'Price': res['score'], 'Technique': name}
    i += 1

results[res_i] = {'Price': measures[1]['Price'], 'Transformation': 'Undersampling 70-30'}
res_i += 1

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [17]:
bal= RandomUnderSampler(random_state=42)
X_train, y_train = balancer.fit_sample(X_train, y_train)

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
measures

{0: {'Price': 14950, 'Technique': 'Undersample 50-50'},
 1: {'Price': 22430, 'Technique': 'Undersample 70-30'},
 2: {'Price': 33550, 'Technique': 'SMOTE 50-50'},
 3: {'Price': 37250, 'Technique': 'SMOTE 70-30'}}

In [18]:
X_train, X_test = normalize(X_train, X_test)

In [19]:
res = aps_classifier_statistics(clf, X_train, X_test, y_train, y_test)
results[res_i] = {'Price': res['score'], 'Transformation': 'Baseline'}
res_i += 1
print(res)

{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.9758125, 'confusion_matrix': array([[15307,   318],
       [   69,   306]]), 'sensibility': 0.816, 'specificity': 0.979648, 'score': 37680}


Drop cd_000. THe results didn't get better, stayed the same

In [6]:
#X_train, X_test = X_train.drop(columns=['cd_000']), X_test.drop(columns=['cd_000'])

In [7]:
#res = aps_classifier_statistics(clf, X_train, X_test, y_train, y_test)
#results[res_i] = {'Price': res['score'], 'Transformation': 'Baseline'}
#res_i += 1
#print(res)

In [8]:
sets = {0: {'bb_000', 'bv_000', 'bu_000', 'cq_000'}, 1: {'ah_000', 'bg_000'}, 2: {'bt_000', 'aa_000'}, 3: {'cf_000', 'co_000', 'ad_000'}}

for group_i in sets:
    group = sets[group_i]
    group_list = list(group)
    pca = PCA(n_components=1).fit(X_train[group_list])
    new = pca.transform(X_train[group_list])
    new_test = pca.transform(X_test[group_list])
    
    new = pd.DataFrame(data=new)
    new_test = pd.DataFrame(data=new_test)
    X_train = pd.concat([X_train, new], axis=1)
    X_test = pd.concat([X_test, new_test], axis=1)
    X_train, X_test = X_train.drop(columns=group_list), X_test.drop(columns=group_list)
    

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
res = aps_classifier_statistics(clf, X_train, X_test, y_train, y_test)
results[res_i] = {'Price': res['score'], 'Transformation': 'Baseline'}
res_i += 1
print(res)